# Pattern matching in Scala

## `match` expression

* `match` expression can be used like switch statement in C.
* `match` does not require `break` statement like in C
(no flow through after first match)
* `match` is an expression that returns the value of the matching
case statement.
* We can use `|` to match multiple values in case condition
* We can use variables in case statement. 
* case statements can have if guards
* case statements can match based on variable types.

In [2]:
println("Enter a arithmetic operator")
val ch = scala.io.StdIn.readChar()

val precedence = ch match {
    case '+' | '-' => 2
    case '*' | '/' | '%' => 1
    // _ can be used like a wild card
    case _ => 3
}

println(s"$ch, $precedence")

Enter a arithmetic operator
+
+, 2


ch: Char = '+'
precedence: Int = 2

In [3]:
// match using variable types
val bag: List[Any] = List[Any]("apple", 10, 100.0, true)

for (item <- bag) {
    item match {
        // match using variable and their types
        case fruit: String => println(fruit)
        // use if guards
        case quantity: Int if quantity > 0 => println(quantity)
        case price: Double => println(price)
        case stockAvailable: Boolean => println(stockAvailable)
        
        // wildcard can be any variable name or underscore
        // * use underscore when  you dont want to use the name
        // in the case expression
        // * use a variable with a name when you want to do something
        // with the name inside the case expression.
        case _ => ()
    }
}


apple
10
100.0
true


bag: List[Any] = List("apple", 10, 100.0, true)

**NOTE**: Variable names used in case statement should always follow lower camel case.
If we use PascalCase, then scala treats those identifiers as constants.

**NOTE**: If case statement's variable name has collision with
names from the import statement, enclose the variable name of the
case statement in back ticks.

**NOTE**: Whenever possible use pattern matching to decide variable
type instead of using `isInstanceOf` multiple times.

* We cannot use generic types in case like `case x: List[String]` due
to type erasure done when compiling to JVM bytecode.

**NOTE**: If a pattern has alternatives using `|`, then pattern expressions
cannot use any other variable other than underscore. Ex: `(0, _) | (_, 0)`

## Matching collections

In [7]:
// arrays
val arr = Array(0, 1)

// we can use extractors in case patterns
arr match {
    // match array of required length
    case Array(x, y) => println(x, y)
    
    // match by array content at position
    case Array(0, _) | Array(_, 0) => println("Array starts or ends with 0")
    
    // match array of any length
    case Array(1, rest @ _*) => println("Array starts with 1")
    case _ => println("not of interest")
}

(0,1)


arr: Array[Int] = Array(0, 1)

In [9]:
// tuples
val tcp_tuple = ("192.168.1.1", "192.168.1.2", 45500, 80, 'T')

// This is just to showcase the match capabilities with Tuple
tcp_tuple match {
    // note _* not allowed for tuples. allowed only for sequences
    case (ip1, ip2, _, _, _) => println(ip1, ip2)
    
    // match by exact values
    case(_, _, _, 80, _:Char) => println("communication with port 80")
    
    // match by exact type
    case x: (String, String, Int, Int, Char) => println(x)
    //case (_:String, _:String, _:Int, _:Int, _:Char) => println("match")
}

(192.168.1.1,192.168.1.2)


tcp_tuple: (String, String, Int, Int, Char) = (
  "192.168.1.1",
  "192.168.1.2",
  45500,
  80,
  'T'
)

In [13]:
// Lists
val simpleList = List(1 ,2, 3, 4, 5)

simpleList match {
    case head :: tail if (tail.last > 5) => println(head, tail)
    case first :: second :: _ :: _ :: 5 :: Nil => println(first, second)
    case _ => ()
}

(1,2)


simpleList: List[Int] = List(1, 2, 3, 4, 5)

In [14]:
// pattern matching in variable declarations
val (x, y) = (5, 10)
println(x, y)

// such pattern matching is also used in for expressions

(5,10)


x: Int = 5
y: Int = 10

## Case classes

* constructor params become `val` fields of classes by default
* Serializable
* comes with equals, hashCode, toString, copy. apply, unapply method
* Since unapply is already provided, they can be used in 
pattern matching

**NOTE**:
* It's recommended to keep all the fields as val in case classes to keep them
immutable.
* Do not inherit a case class from another case class. Always keep
the base class as plain classes

In [16]:
case class Person(name: String, age: Int)

// notice we need not instantiate case classes with new
val john = Person("John", 25)
val jane = Person("Jane", 25)

//toString
println(john)

// equals
println(john == jane)

john match {
    case Person("John", _) => println("Person named John")
    case _ => println("Not named John")
}

// using keyword parameters aka named parameters,
// we can customize the copy.
val johnSenior = john.copy(age=60)
println(johnSenior)

Person(John,25)
false
Person named John
Person(John,60)


defined class Person
john: Person = Person("John", 25)
jane: Person = Person("Jane", 25)
johnSenior: Person = Person("John", 60)

In [16]:
// infix notation can be used with case classes during pattern match
// Infix notation works with any unapply method that returns a pair(2-tuple)
// in lists where we matched head :: tail, here :: is a case class
// Use infix notation only when case class names are operators

* When a class is declared `sealed` all of its subclasses should reside
in the same scala source file. Its recommended for case classes to extend
a sealed class or trait.

## `Option` type

* Option is a generic type that can be either Some(value) or None.
* `isEmpty`, `isDefined`, `get` are some useful methods on options.
* Its often recommended to treat Option as a List with either 0 or 1
elements. Hence we could use methods like map, foreach, filter, flatMap
for operating on Options type.

## Partial Functions

Partial function is a block that contains a collection of 
case expressions but that does not handle all cases.

In [19]:
val pf: PartialFunction[Char, Int] = {
    case '+' => 1
    case '-' => -1
}
// called like a function
pf('-')

// raises MatchError when input is not handled
// we could use isDefinedAt() to check if partial function is capable of
// handling the input
pf.isDefinedAt('*')

// partial functio to function using lift method
// that returns Option[T]
// any function returning Option[T] is converted to
// partial function using Function.unlift
val f = pf.lift
println(f)

<function1>


pf: PartialFunction[Char, Int] = <function1>
res18_1: Int = -1
res18_2: Boolean = false
f: Char => Option[Int] = <function1>